In [2]:
import pyranges as pr
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
import gzip
import pandas as pd
import random
import json

from pandasql import sqldf
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
from collections import deque
import math
import re

In [3]:
cofactor_path = '/Users/ugur_dura/Desktop/IN2393-Machine Learning for Regulatory Genomics/Project/promoter_sequence_identification/data/raw_data/Haberle_COF_STAP_oligos.xlsx'
fasta_path = '/Users/ugur_dura/Desktop/IN2393-Machine Learning for Regulatory Genomics/Project/promoter_sequence_identification/Trial_Scripts/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fa'




In [4]:
cofactor_read_dataframe = pd.read_excel(cofactor_path)

In [5]:
print(cofactor_read_dataframe['full_name'])

0                                                 DSCP
1              chr2LHet_131063_131196_131130_+_STAPseq
2              chr2LHet_151154_151287_151221_+_STAPseq
3              chr2LHet_166318_166451_166385_+_STAPseq
4           chr2LHet_167196_167329_167263_+_embryoCAGE
                             ...                      
71995    chrYHet_81142_81275_81209_+_FlyBaseAnnotation
71996                                      ftz_DPE_mut
71997                                     ftz_TATA_mut
71998                                       gt_DPE_mut
71999                                      gt_TATA_mut
Name: full_name, Length: 72000, dtype: object


In [6]:
sentence = "chr2LHet_131063_131196_131130_+_STAPseq"
pattern = r"^(chr\w+?)(?:Het)?_(\d+)_(\d+)_(\d+)_(\+|-)_\w+$"
match = re.search(pattern, sentence)

if match:
    chromosome = match.group(1)
    start_index = int(match.group(2))
    end_index = int(match.group(3))
    tss_start_index = int(match.group(4))
    strand = match.group(5)

    print("Chromosome:", chromosome)
    print("Start Index:", start_index)
    print("End Index:", end_index)
    print("TSS Start Index:", tss_start_index)
    print("Strand:", strand)

Chromosome: chr2L
Start Index: 131063
End Index: 131196
TSS Start Index: 131130
Strand: +


In [7]:
columns = cofactor_read_dataframe.columns

cofactors =columns[2:].to_list()

print(cofactors)




['p65', 'Nej_p300', 'Med25', 'Lpt', 'Med15', 'CG7154_Brd9', 'fs1h_Brd4', 'trr', 'trx', 'gfzf', 'Chro', 'CG15356_EMSY', 'Brd8', 'Mof', 'GFP']


In [8]:
id = []
chromosome_keys = []
start_index = []
end_index = []
TSS_start_index = []
strand = []
expression_list_oflist = []

pattern = r"^(chr\w+?)(?:Het)?_(\d+)_(\d+)_(\d+)_(\+|-)_\w+$"

for i in range(len(cofactor_read_dataframe['full_name'])):
    sentence = cofactor_read_dataframe['full_name'][i]
    match = re.search(pattern, sentence)

    if match:
        if match.group(1) != 'chrU' and match.group(1) != 'chrUextra' and match.group(1) !=  'chrM':
            id.append(sentence)
            chromosome_keys.append(match.group(1))
            start_index.append(int(match.group(2)))
            end_index.append(int(match.group(3)))
            TSS_start_index .append(int(match.group(4)))
            strand.append(match.group(5))
            expressions = []
            for cofactor in cofactors:
                expression_level = cofactor_read_dataframe[cofactor][i]
                expressions.append(expression_level)
        
            expression_list_oflist.append(expressions)



        






In [9]:
print(chromosome_keys)

print(any("chrUextra" in word for word in chromosome_keys))

['chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 'chr2L', 

In [10]:
fasta_file = open(fasta_path, 'r')
fasta_lines = fasta_file.readlines()

pattern = r'>\s*([^ ]+)\s+.*chromosome\s+([^\s]+)'
descriptions = {}

for line in fasta_lines:
    if line.startswith('>'):
        if not "sequence" in line:
            print(line)
            match = re.search(pattern, line)
            if match:
                chromosome = "chr" + match.group(2)
                value = match.group(1)
 
                descriptions[chromosome] = value


# descriptions['chrM'] = 'NC_024511.2'
# descriptions['chrU']= 'NW_007931084.1'
# descriptions['chrUextra']= 'NW_007931084.1'

>NC_004354.4 Drosophila melanogaster chromosome X

>NT_033779.5 Drosophila melanogaster chromosome 2L

>NT_033778.4 Drosophila melanogaster chromosome 2R

>NT_037436.4 Drosophila melanogaster chromosome 3L

>NT_033777.3 Drosophila melanogaster chromosome 3R

>NC_004353.4 Drosophila melanogaster chromosome 4

>NC_024512.1 Drosophila melanogaster chromosome Y

>NC_024511.2 Drosophila melanogaster mitochondrion, complete genome



In [11]:
records = SeqIO.to_dict(SeqIO.parse(open(fasta_path), 'fasta'))



In [40]:
values=[(key,value) for key, value in records.items() if 'NC' in key or 'NT' in key]
records_dict = dict(values)
   

In [41]:
print(records_dict)

{'NC_004354.4': SeqRecord(seq=Seq('GAATTCGTCAGAAATGAgctaaacaaatttaaatcattaaatgcGAGCGGCGAA...tac'), id='NC_004354.4', name='NC_004354.4', description='NC_004354.4 Drosophila melanogaster chromosome X', dbxrefs=[]), 'NT_033779.5': SeqRecord(seq=Seq('cgacaatgcacgacagaggaagcagaacagatatttagattgcctctcattttc...GAG'), id='NT_033779.5', name='NT_033779.5', description='NT_033779.5 Drosophila melanogaster chromosome 2L', dbxrefs=[]), 'NT_033778.4': SeqRecord(seq=Seq('Ctcaagataccttctacagattatttaaagctagtgcacaacaacaataaattg...TTC'), id='NT_033778.4', name='NT_033778.4', description='NT_033778.4 Drosophila melanogaster chromosome 2R', dbxrefs=[]), 'NT_037436.4': SeqRecord(seq=Seq('tagggagaaatatgatcgcgtatgcgagagtagtgccaacatattgtgctcttt...TAT'), id='NT_037436.4', name='NT_037436.4', description='NT_037436.4 Drosophila melanogaster chromosome 3L', dbxrefs=[]), 'NT_033777.3': SeqRecord(seq=Seq('acgggaccgagtatagtaccagtacgcgaccagtacgggagcagtacggaacca...TTC'), id='NT_033777.3', name='NT_033777.3', descript

In [42]:
sequence = str((records_dict['NC_004354.4']).seq)

# Print the sequence
print(sequence[0:10])

GAATTCGTCA


In [16]:
print(descriptions)

def reverse_complementary_sequence(dna_sequence):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    reverse_complementary_seq = ''.join(complement[base] for base in reversed(dna_sequence))
    return reverse_complementary_seq

 

{'chrX': 'NC_004354.4', 'chr2L': 'NT_033779.5', 'chr2R': 'NT_033778.4', 'chr3L': 'NT_037436.4', 'chr3R': 'NT_033777.3', 'chr4': 'NC_004353.4', 'chrY': 'NC_024512.1'}


In [20]:
list_tuple_TSS = []


for i in range(len(id)):
    id_= id[i]
    chromosome_ = chromosome_keys[i]
    start_index_ = start_index[i]
    end_index_ = end_index[i]
    strand_ = strand[i]
    
    gene_ID = descriptions[chromosome_]
    sequence = records_dict[gene_ID].seq

    sequence = sequence[start_index_:end_index_+1]
    sequence = sequence.upper()

    if strand == "-":
        sequence = reverse_complementary_sequence(sequence)

    
    new_row  = ("1",
                chromosome_, 
                str(start_index_), 
                str(end_index_), 
                strand_, 
                id_, 
                sequence, 
                str(len(sequence)))

    list_tuple_TSS.append(new_row)

        


    


In [21]:
expression_dataframe= pd.DataFrame(list_tuple_TSS, columns=['TSS','seqnames', 'start', 'end', 'strand', 'ID', 'sequence', 'sequence_len'])



In [23]:
print(expression_dataframe)

      TSS seqnames   start     end strand   
0       1    chr2L  131063  131196      +  \
1       1    chr2L  151154  151287      +   
2       1    chr2L  166318  166451      +   
3       1    chr2L  167196  167329      +   
4       1    chr2L  168943  169076      -   
...    ..      ...     ...     ...    ...   
71625   1     chrY  344367  344500      -   
71626   1     chrY    4518    4651      +   
71627   1     chrY    4544    4677      -   
71628   1     chrY   59388   59521      +   
71629   1     chrY   81142   81275      +   

                                                  ID   
0            chr2LHet_131063_131196_131130_+_STAPseq  \
1            chr2LHet_151154_151287_151221_+_STAPseq   
2            chr2LHet_166318_166451_166385_+_STAPseq   
3         chr2LHet_167196_167329_167263_+_embryoCAGE   
4          chr2LHet_168943_169076_169010_-_adultCAGE   
...                                              ...   
71625         chrYHet_344367_344500_344434_-_STAPseq   
71626      

In [ ]:
expression_dataframe[cofactors] = expression_list_oflist

In [ ]:
print(expression_dataframe)

      TSS seqnames   start     end strand   
0       1    chr2L  131063  131196      +  \
1       1    chr2L  151154  151287      +   
2       1    chr2L  166318  166451      +   
3       1    chr2L  167196  167329      +   
4       1    chr2L  168943  169076      -   
...    ..      ...     ...     ...    ...   
71990   1     chrY  344367  344500      -   
71991   1     chrY    4518    4651      +   
71992   1     chrY    4544    4677      -   
71993   1     chrY   59388   59521      +   
71994   1     chrY   81142   81275      +   

                                                  ID   
0            chr2LHet_131063_131196_131130_+_STAPseq  \
1            chr2LHet_151154_151287_151221_+_STAPseq   
2            chr2LHet_166318_166451_166385_+_STAPseq   
3         chr2LHet_167196_167329_167263_+_embryoCAGE   
4          chr2LHet_168943_169076_169010_-_adultCAGE   
...                                              ...   
71990         chrYHet_344367_344500_344434_-_STAPseq   
71991      

In [ ]:
expression_dataframe.to_csv('cofactor_expression_data.csv',index=True)